## 1. 简化计算单细胞lcczscore
* lcczscore:  最大连通分支上节点数量的显著性
* weighted-lcczscore: 最大连通分支上表达值的显著性
* expressmean-lcczscore: 最大连通分支上平均每个基因表达的显著性
* 
      1.1 pip>1时，正常随机 ，只记录单细胞中基因数量有变化的点  【高扰动】
      1.2 计算“所有gene”的lcczscore和weighted-lcczscore，expressmean-lcczscore 【低扰动】

## 2. 根据 lcczscore pip 计算得到的结果提取每个细胞的6个点
对于每个细胞
* 统计pip大于1 的lcczscore最大值，以及该位置对应的weightedlcczscore，expresszscore
* 统计pip=0处(所有基因) 的lcczscore，以及该位置对应的weightedlcczscore，expresszscore

## 1. 计算lcczscore，weighted-lcczscore,expressmean-lcczscore

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random
import numpy as np
import scanpy as sc
import math
import time

import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")

In [2]:
import os
def makedir(folder_path):
    # 检查文件夹是否已经存在
    if not os.path.exists(folder_path):
        # 文件夹不存在时创建
        os.makedirs(folder_path)
        print(f"文件夹 '{folder_path}' 已创建")
    else:
        print(f"文件夹 '{folder_path}' 已存在，不进行任何处理")

In [3]:
'''读取背景网络'''
def openPPI(filename):
    '''
        打开PPI
        文件格式 gene1_name gene1_id gene2_name gene2_id
        返回值：网络[节点是gene name]
    '''
    G = nx.Graph()
    a=open(filename,"r")
    next(a)

    for i in a:
        n=i.strip().split("\t")
        G.add_edge(n[0],n[3])
        G.add_node(n[0])
        G.add_node(n[3])
    a.close()
    return G

G = openPPI(r"..\data\network(process_id).txt")

In [4]:
# 计算最大连通分支
def Lcc(G,genename):
    '''
        输入图网络和一个列表，计算最大连通分支lcc
        G：图，这里图中的节点是基因name
        genename：用于计算lcc的genename列表
    '''

    g = nx.subgraph(G,genename)
    if len(genename)==0:
        largest = 0
        l = []
        return l,largest
    else:
        try:
            l = max(nx.connected_components(g),key=len)  #如果随机选取的gene没有连通分支的话，会报错，添加判断条件
            largest = len(l)     #最大联通分支
        except ValueError as e:
            l = genename[0]    # 假设genename列表中的第一个基因为最大连通分支
            largest = 1        #设置最大联通分支数为1
        return list(l),largest

In [5]:
## 计算一组基因集合的表达值均值
def expressValue(genelist,annData_df,cell):
    '''
    :param genelist: 要计算的gene列表
    :param annData_df: 单细胞表达数据的df
    :param cell:  第几个细胞
    :return:
    '''

    # cell_expressedvalue = annData_df.loc[annData_df.index[cell],genelist]
    cell_expressedvalue = annData_df.loc[cell,genelist]
    if len(cell_expressedvalue)==0:
        return 0,0
    else:
        return sum(cell_expressedvalue), sum(cell_expressedvalue)/len(cell_expressedvalue)

In [6]:
# 计算最大连通分支，并返回最大连通分支上的基因表达值之和、均值
def lccExpressedValue(G,genename,annData_df,cell):
    '''
    :param g: 选定的背景网络
    :param genename:
    :param annData_df:
    :param cell:
    :return:
    '''

    lccgenelist, largest = Lcc(G,genename)
    lcc_expresssum,lcc_expressmean = expressValue(lccgenelist,annData_df,cell)
    return lcc_expresssum,lcc_expressmean

In [7]:
def LccExpress_mean(g,genelist,ran,annData_df,cell):
    '''
    每次从网络G的所有基因中随机选择genelist数量的gene，计算均值和标准差
    g:背景网络
    genelist：
    ran：随机次数
    返回：表达值之和、表达值均值的多次随机的均值和标准差
    '''

    all_genes = g.nodes()  # 背景网络中的gene

    #做随机实验
    l_list = []         #lcc列表
    l1_list = []         #lcc列表
    l2_list = []         #lcc列表
    for j in range(ran):
        black_nodes = random.sample(all_genes,len(genelist))
        l,largest_ran = Lcc(g,black_nodes)
        lcc_expresssum,lcc_expressmean = lccExpressedValue(g,black_nodes,annData_df,cell)

        l_list.append(largest_ran)
        l1_list.append(lcc_expresssum)
        l2_list.append(lcc_expressmean)

    #计算lcczscore
    lcc_mean = np.mean(l_list)
    lcc_std  = np.std(l_list)

    lcc1_mean = np.mean(l1_list)
    lcc1_std  = np.std(l1_list)

    lcc2_mean = np.mean(l2_list)
    lcc2_std  = np.std(l2_list)

    return lcc_mean,lcc_std,lcc1_mean,lcc1_std,lcc2_mean,lcc2_std

In [8]:
# 计算lcczscore
def lccZscore(Lcc,lcc_mean,lcc_std):
    if lcc_std == 0:
        zscore = 0
    else:
        zscore = (Lcc - lcc_mean)/lcc_std
        zscore = round(zscore,6)
    return zscore


In [9]:
# 计算某一组基因集合的lccexpresszscore
# G：整个背景网络
# g：随机选择基因集合的背景
def gensetLccExpressZscore(G,g,genelist,ran,annData_df,cell):
    l,lcc = Lcc(G,genelist)
    lcc_expresssum,lcc_expressmean = lccExpressedValue(G,genelist,annData_df,cell)
    lcc_mean,lcc_std,lcc_expresssum_mean,lcc_expresssum_std,lcc_expressmean_mean,lcc_expressmean_std = \
        LccExpress_mean(g,genelist,ran,annData_df,cell)

    lcc_zscore = lccZscore(lcc,lcc_mean,lcc_std)
    lcc_expresssum_zscore = lccZscore(lcc_expresssum,lcc_expresssum_mean,lcc_expresssum_std)
    lcc_expressmean_zscore = lccZscore(lcc_expressmean,lcc_expressmean_mean,lcc_expressmean_mean)

    return lcc_zscore,lcc_expresssum_zscore,lcc_expressmean_zscore

### 采样策略：等pip采样



In [10]:
# 读取asthma gwas数据中所有基因
trait = pd.read_csv(r"..\data\poly_posterior_prior_gene_new\29_new.txt",sep = "\t")
trait_genelist = list(set(list(trait["gene name"])))   # 性状相关基因 12676
# gene-pip排序
grouped_df = trait.groupby('gene name').sum()['posterior']
grouped_dfsorted = grouped_df.sort_values(ascending=False)
grouped_dfsorted = pd.DataFrame(grouped_dfsorted[1:]) #删除“-”
grouped_dfsorted["gene name"] = grouped_dfsorted.index

In [11]:
def samplePip(grouped_dfsorted):
    '''
    采样策略：
            [0)
            [1~4.238)  全取  227次

    :param grouped_dfsorted:
    :return:  pip的分界线
    '''
    temp1 = [0]
    temp4 = sorted(list(set(grouped_dfsorted[grouped_dfsorted['posterior']>=1]['posterior'].sort_values())))
    temp5 = [max(grouped_dfsorted['posterior']) + 0.1]
    binspip = temp1 +  temp4 + temp5
    return binspip

# 只选特定的一些点来计算
binspip = samplePip(grouped_dfsorted)
grouped_dfsorted['Category'] = pd.cut(grouped_dfsorted['posterior'], bins=binspip, right=False)

In [ ]:
tissue = "Prostate"
# 读取TS FACS单细胞数据
filePath = r"..\data\TS FACS\TS_%s.h5ad"%tissue
annData = sc.read_h5ad(filePath)
# 按照指定列进行分组
obs_df = pd.DataFrame(annData.obs)
grouped_adata = obs_df.groupby("cell_ontology_class")
# 使用value_counts()方法计算每个分组中的细胞数量
group_counts = obs_df["cell_ontology_class"].value_counts()
# 按细胞数量逆序排列分组
sorted_groups = group_counts.sort_values()
print(sorted_groups)
celltypelist = list(sorted_groups.keys())

In [12]:
num = 0
for celltype in celltypelist[0:10]:
    num +=1
    print(celltype)
    print(str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
    # 创建结果文件夹
    makedir(r"..\results\lcczscore-pip\%s\%s"%(tissue,celltype))
    logpath = r"..\results\lcczscore-pip\%s\%s\0log.txt"%(tissue,celltype)
    with open(logpath, "w") as f:  # 先清空
        f.truncate(0)
    # 取出 'cd4-positive alpha-beta t cell' 对应的组的数据
    subset_annData = annData[annData.obs['cell_ontology_class'] == celltype, :]
    subset_annData_df = subset_annData.to_df()
    # 有表达的基因
    expressedGenes = subset_annData_df.apply(lambda row: subset_annData_df.columns[row.to_numpy().nonzero()[0]].tolist(), axis=1)
    celllist = list(expressedGenes.index)
    # 计算每个细胞中的结果
    count = 0

    for cell in celllist:   #cell是细胞名字
        count+=1
        with open(logpath, 'a') as r:
            r.write("第" + str(count) + "/" + str(len(celllist)) +"个细胞:"+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))+"\n")
        genelist = []
        result_list = []

        # 单细胞有表达的网络基因
        singlecell_network_genelist = list(set(expressedGenes[cell]) & set(list(G.nodes())))
        # 构建单细胞子网络
        g = nx.subgraph(G,singlecell_network_genelist)
        # 细胞有表达的asthma相关基因
        # 每个细胞都采样426次，但是每次采样中加入的基因数目不同，有的采样可能不加入新基因
        allgene = grouped_dfsorted[grouped_dfsorted["gene name"].isin(singlecell_network_genelist)]
        allgene['Left_Boundary'] = allgene['Category'].apply(lambda x: x.left)
        gene_df= allgene.groupby('Left_Boundary')["gene name"].agg(list).reset_index()
        gene_df_sorted = gene_df.iloc[::-1]

        for i in range(len(gene_df_sorted)):
            if len(gene_df_sorted.iloc[i]["gene name"]) != 0:  # 只有当该细胞中基因数量发生变化时，才计算
                genelist.extend(gene_df_sorted.iloc[i]["gene name"])
                l,lcc = Lcc(G,genelist)
                # lcc_expresszscore
                lcczscore,lcc_expresssum_zscore,lcc_expressmean_zscore = \
                    gensetLccExpressZscore(G,g,genelist,50,subset_annData_df,cell)  # 随机50次
                result_list.append([gene_df_sorted.iloc[i]["Left_Boundary"],len(genelist),lcc,lcczscore,lcc_expresssum_zscore,lcc_expressmean_zscore])

        df_singcell = pd.DataFrame(result_list)
        df_singcell.columns = ["pip_Boundary","genenum","lcc","lcczscore","lcc_expresssum_zscore","lcc_expressmean_zscore"]
        df_singcell.to_csv(r"..\results\lcczscore-pip\%s\%s\%s.csv"%(tissue,celltype,cell))
        r.close()

mast cell                                      9
sperm                                         11
neutrophil                                    16
stromal cell                                  32
monocyte                                      33
erythroid progenitor                          86
nkt cell                                     175
hillock cell of prostate epithelium          211
fibroblast                                   223
cd8b-positive nk t cell                      255
smooth muscle cell                           285
hillock-club cell of prostate epithelium     290
macrophage                                   317
myeloid cell                                 365
endothelial cell                             585
t cell                                       586
club cell of prostate epithelium             788
cd8-positive, alpha-beta t cell             1081
luminal cell of prostate epithelium         1092
basal cell of prostate epithelium           3298
epithelial cell     

## 2. 根据 lcczscore pip 计算得到的结果提取每个细胞的6个点
    * 统计pip大于1 的lcczscore最大值，以及该位置对应的weightedlcczscore，expresszscore
    * 统计pip=0处(所有基因) 的lcczscore，以及该位置对应的weightedlcczscore，expresszscore


In [ ]:
# 遍历文件夹中的指定格式的文件
def openFolderAll(folder_path, desired_extension):
    filelist = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith(desired_extension):
            filelist.append(file_path)
    return filelist


In [ ]:
boundary = 1
tissuelist = os.listdir(r"..\results\lcczscore-pip")
for tissue in tissuelist[20:]:
    # 创建结果文件夹
    makedir(r"..\results\lcczscore-pip-cell-sixpoints\%s" % tissue)

    ## 目前文件夹中结果的细胞类型
    celltypelist = os.listdir(r"..\results\lcczscore-pip\%s" % tissue)
    for celltype in celltypelist:
        # 读取文件夹下的所有文件
        folder_path = r"..\results\lcczscore-pip\%s\%s" % (tissue, celltype)
        desired_extension = '.csv'
        filelist = openFolderAll(folder_path, desired_extension)

        celltype_result = []
        for file in filelist:
            cellname = file.split("\\")[-1].split(".")[0]
            result_singcell = pd.read_csv(file, index_col=0)

            if len(result_singcell) <= 6:
                continue
            # 条件筛选删除小于边界值的行
            result_singcell = result_singcell[result_singcell["genenum"] >= 5].reset_index(drop=True)
            # 后验概率>1的最大值
            if len(result_singcell[result_singcell["pip_Boundary"] >= boundary]) != 0:
                # pip大于1 的lcczscore最大值，以及该位置对应的lccexpresssum
                maxlcczscore = max(result_singcell[result_singcell["pip_Boundary"] >= boundary]["lcczscore"])
                index_maxlcczscore = result_singcell[result_singcell["pip_Boundary"] >= boundary]["lcczscore"].idxmax()
                maxpip = result_singcell.iloc[index_maxlcczscore]["pip_Boundary"]
                maxlcc_expresszscore = result_singcell.iloc[index_maxlcczscore]["lcc_expresssum_zscore"]
                maxlcc_expressmeanzscore = result_singcell.iloc[index_maxlcczscore]["lcc_expressmean_zscore"]
                maxlcc_genenum = result_singcell.iloc[index_maxlcczscore]["genenum"] + 5
                maxlcc_lcc = result_singcell.iloc[index_maxlcczscore]["lcc"]

                # pip=0处 的lcczscore，以及该位置对应的lccexpresssum
                allgene_lcczscore = result_singcell.iloc[len(result_singcell) - 1]["lcczscore"]
                allgene_lccexpresszscore = result_singcell.iloc[len(result_singcell) - 1]["lcc_expresssum_zscore"]
                allgene_lccexpressmeanzscore = result_singcell.iloc[len(result_singcell) - 1]["lcc_expressmean_zscore"]
                celltype_result.append(
                    [cellname, index_maxlcczscore, maxpip, maxlcczscore, maxlcc_expresszscore, maxlcc_expressmeanzscore,
                     maxlcc_genenum, maxlcc_lcc,
                     allgene_lcczscore, allgene_lccexpresszscore, allgene_lccexpressmeanzscore])
        df_result = pd.DataFrame(celltype_result)
        print(celltype + "  finish!")
        if len(df_result) > 0:
            df_result.columns = ["cellname", "index_maxlcczscore", "maxpip", "maxlcczscore", "maxlcc_expresszscore",
                                 "maxlcc_expressmeanzscore", "maxlcc_genenum", "maxlcc_lcc",
                                 "allgene_lcczscore", "allgene_lccexpresszscore", "allgene_lccexpressmeanzscore"]
            df_result.to_csv(r"..\results\lcczscore-pip-cell-sixpoints\%s\%s.csv" % (tissue, celltype))

    print("Finish!")